## Intent classifier trainer

* Dataset: student_queries.csv file with synthetic messages, each labeled with one of the intent categories, such as: Course Information, Enrollment / Course Registration, Withdrawal or Drop Course, Scholarship/Financial Aid, etc.
* Task: Single-label classification to predict the intent behind the student's query.
* Model: Use BertForSequenceClassification, fine-tuned on the intent dataset.
* Goal: To determine the purpose of a query.
* The trained model is saved to models/intentClassifier.

In [1]:
intent_labels = [
        "Course Information",
        "Enrollment / Course Registration",
        "Withdrawal or Drop Course",
        "Access Issues (portal/login)",
        "Technical Support",
        "Tuition/Fees Inquiry",
        "Scholarship/Financial Aid",
        "Mental Health Concerns",
        "Stress or Burnout",
        "Bullying or Harassment",
        "Administrative Support",
        "Campus Facilities",
        "Housing/Accommodation",
        "Extracurricular Activities",
        "General Complaint"
    ]

In [ ]:
import pandas as pd

label_to_id = {label: idx for idx, label in enumerate(intent_labels)}
id_to_label = {idx: label for idx, label in enumerate(intent_labels)}

df = pd.read_csv("../../data/student_queries.csv")

# Encode labels
df["label"] = df["intent"].map(label_to_id)
df = df.dropna(subset=["label"])

# convert label to int (from float due to NaN)
df["label"] = df["label"].astype(int)

In [7]:
df.head()

,datetime,student,question,intent,is_distressed,label
0,2025-07-04 14:07:23,Shannon Austin,What topics will be covered in the AI course?,Course Information,False,0
1,2025-07-04 16:13:28,Stephanie Calhoun,I'm feeling really overwhelmed lately.,Mental Health Concerns,True,7
2,2025-07-04 16:19:28,Kevin Garcia,Who do I contact for transcript requests?,Administrative Support,False,10
3,2025-07-04 16:55:20,Lisa Duran,I have a complaint about the cafeteria service.,General Complaint,True,14
4,2025-07-04 17:08:35,Jeff Rangel,Who do I contact for transcript requests?,Administrative Support,False,10


In [8]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[["question", "label"]])
test_dataset = Dataset.from_pandas(test_df[["question", "label"]])

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["question"], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Model setup
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(intent_labels)
)

# Metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/Users/bennyxiong/Documents/Source/ML/CSCN8010_FinalProject/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at /Users/bennyxiong/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/bennyxiong/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/tokenizer_config.json
loading configuration file config.json from cache at /Users/bennyxiong/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertCon

In [10]:
# Train the model
trainer.train()

# Evaluate on test set
trainer.evaluate()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, question. If __index_level_0__, question are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/bennyxiong/Documents/Source/ML/CSCN8010_FinalProject/.venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2460
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1540
  Number of trainable parameters = 109493775
/Users/bennyxiong/Documents/Source/ML/

{'loss': 1.8348, 'learning_rate': 1.8701298701298704e-05, 'epoch': 0.65}


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, question. If __index_level_0__, question are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 615
  Batch size = 64










                                        

                                         
  0%|          | 0/1540 [04:42<?, ?it/s]          

Saving model checkpoint to ./results/checkpoint-154
Configuration saved in ./results/checkpoint-154/config.json


{'eval_loss': 0.2902831733226776, 'eval_accuracy': 0.9886178861788618, 'eval_f1': 0.9885946487373333, 'eval_precision': 0.9888852875239715, 'eval_recall': 0.9886178861788618, 'eval_runtime': 2.5486, 'eval_samples_per_second': 241.313, 'eval_steps_per_second': 3.924, 'epoch': 1.0}


Model weights saved in ./results/checkpoint-154/pytorch_model.bin
/Users/bennyxiong/Documents/Source/ML/CSCN8010_FinalProject/.venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                                        
  0%|          | 0/1540 [04:58<?, ?it/s]          

...

{'eval_loss': 0.0399785116314888, 'eval_accuracy': 0.9902439024390244, 'eval_f1': 0.9902228425898717, 'eval_precision': 0.9903213317847464, 'eval_recall': 0.9902439024390244, 'eval_runtime': 2.555, 'eval_samples_per_second': 240.707, 'eval_steps_per_second': 3.914, 'epoch': 4.0}


Model weights saved in ./results/checkpoint-616/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-308 (score: 0.9918486169913014).
                                        
 40%|████      | 616/1540 [03:43<05:35,  2.76it/s]
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, question. If __index_level_0__, question are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 615
  Batch size = 64
/Users/bennyxiong/Documents/Source/ML/CSCN8010_FinalProject/.venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'train_runtime': 223.4306, 'train_samples_per_second': 110.101, 'train_steps_per_second': 6.893, 'train_loss': 0.39175636314049167, 'epoch': 4.0}


100%|██████████| 10/10 [00:02<00:00,  4.38it/s]


{'eval_loss': 0.05840090662240982,
 'eval_accuracy': 0.991869918699187,
 'eval_f1': 0.9918486169913014,
 'eval_precision': 0.99198606271777,
 'eval_recall': 0.991869918699187,
 'eval_runtime': 2.5571,
 'eval_samples_per_second': 240.503,
 'eval_steps_per_second': 3.911,
 'epoch': 4.0}

In [17]:
trainer.save_model("../../models/intentClassifier")
tokenizer.save_pretrained("../../models/intentClassifier")

Saving model checkpoint to ../../models/intentClassifier
Configuration saved in ../../models/intentClassifier/config.json
Model weights saved in ../../models/intentClassifier/pytorch_model.bin
tokenizer config file saved in ../../models/intentClassifier/tokenizer_config.json
Special tokens file saved in ../../models/intentClassifier/special_tokens_map.json


('../../models/intentClassifier/tokenizer_config.json',
 '../../models/intentClassifier/special_tokens_map.json',
 '../../models/intentClassifier/vocab.txt',
 '../../models/intentClassifier/added_tokens.json')

In [12]:
# Inference example
def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class_id = outputs.logits.argmax().item()
    return intent_labels[predicted_class_id]

In [13]:
# Example usage
print(predict_intent("How do I apply for scholarships?"))
print(predict_intent("I have a complaint about the cafeteria service?"))
print(predict_intent("Are there any upcoming student events"))

Scholarship/Financial Aid
General Complaint
Extracurricular Activities


In [16]:
print(predict_intent("When is the tuition payment deadline?"))
print(predict_intent("Hi, I'm trying to figure out how to pay my tuition fees."))

Tuition/Fees Inquiry
Tuition/Fees Inquiry
